In [1]:
!pip install -q -U torch datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate peft bitsandbytes trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 

In [2]:
from google.colab import userdata


from huggingface_hub import login
login(token=userdata.get('HF_Token')) # Replace HF access token

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Import Dependencies

In [3]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

Loading Quantized Mistral model

In [4]:
#################################################################
# Tokenizer
#################################################################

model_name='mistralai/Mistral-7B-Instruct-v0.2'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Number of Trainable Parameters

In [5]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


Building Pipeline

In [6]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=2000,
)

In [7]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

Chunking Document and Vector Database

In [18]:
from langchain_core.documents import Document

with open('/content/context_doc.txt', 'r') as file:
    # Read the entire file content
    txt_doc = file.read()

doc = [Document(page_content=txt_doc)]


text_splitter = CharacterTextSplitter(chunk_size=256, chunk_overlap=0)
chunked_documents = text_splitter.split_documents(doc)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

Prompt Template and LLM Chain

In [19]:
prompt_template = """
### [INST] Instruction: Answer the question based on your hardware and Internet of Things knowledge. Here is context to help:

{context}

### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [20]:
## Example - No RAG

llm_chain.invoke({"context": "", "question": "What is the use of Stepper Motor in an IoT kit? Write the description, applications and technical specifications."})



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


KeyboardInterrupt: 

RAG Chain

In [21]:
torch.cuda.empty_cache()

In [22]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("What is the use of Stepper Motor in an IoT kit? Write the description, applications and technical specifications.")


print("Context:\n\n", result['context'],"\n\n")
print("Output:\n\n", result['text'],"\n\n")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Context:

 [Document(page_content='Stepper Motor:\nDescription:\nA stepper motor is an electromechanical device that converts electrical pulses into precise mechanical movements. Unlike traditional DC or AC motors, which rotate continuously when powered, stepper motors move in discrete steps, making them ideal for applications requiring precise positioning and control. Stepper motors consist of multiple coils arranged in a specific pattern around a central rotor. By energizing these coils in a specific sequence, the motor advances step by step, providing accurate control over rotation.\nApplications:\nStepper motors find widespread use in various industries and applications, including robotics, CNC machines, 3D printers, automated manufacturing systems, camera positioning systems, and medical devices. Their ability to precisely control position and speed makes them indispensable in tasks requiring accurate motion control, such as in industrial automation for assembly lines or in consum

In [23]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("What is the use of hiletgo gy-521 module in an IoT kit? Write the description, applications and technical specifications.")


print("Context:\n\n", result['context'],"\n\n")
print("Output:\n\n", result['text'],"\n\n")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Context:

 [Document(page_content='HiLetgo GY-521 module:\nDescription:\nThe HiLetgo GY-521 module is a versatile sensor module designed for motion tracking and orientation sensing. It features a built-in accelerometer and gyroscope, along with a temperature sensor, all housed on a small breakout board. With its compact size and easy-to-use interface, it is widely used in various projects requiring precise motion detection and measurement.\nApplications:\nThe GY-521 module finds applications in a wide range of fields, including robotics, drone navigation, virtual reality systems, and motion-controlled gaming devices. It is also commonly used in wearable technology such as fitness trackers and smartwatches for monitoring movement and orientation. Additionally, it is employed in educational settings for teaching concepts related to inertial sensing and motion analysis.\nTechnical Specifications:\nAccelerometer Range: ±2g, ±4g, ±8g, or ±16g selectable\nGyroscope Range: ±250, ±500, ±1000, 

In [24]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("What is the use of Servo Motor SG90 in an IoT kit? Write the description, applications and technical specifications.")


print("Context:\n\n", result['context'],"\n\n")
print("Output:\n\n", result['text'],"\n\n")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Context:

 [Document(page_content='Servo Motor SG90:\nDescription:\nThe SG90 servo motor is a compact and lightweight device known for its precision and versatility in various robotic and electronic projects. It features a small form factor, making it suitable for applications where space is limited. The motor incorporates gears and a control circuit to precisely position the output shaft to a desired angle within a 180-degree range.\nApplications:\nThe SG90 servo motor finds applications in a wide range of fields, including robotics, remote-controlled vehicles, model airplanes, and various DIY projects. It is commonly used in small-scale robotic arms, pan-tilt camera mounts, robotic toys, and other projects requiring precise control of angular motion. Additionally, it can be employed in educational settings to teach principles of robotics and mechatronics.\nTechnical Specifications:\nOperating Voltage: 4.8V - 6.0V\nStall Torque: 1.8 kg/cm (at 4.8V), 2.2 kg/cm (at 6.0V)\nOperating Spee

In [25]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("What is the use of Water Level Detection Sensor Module in an IoT kit? Write the description, applications and technical specifications.")


print("Context:\n\n", result['context'],"\n\n")
print("Output:\n\n", result['text'],"\n\n")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Context:

 [Document(page_content="Water Level Detection Sensor Module:\nDescription:\nThe Water Level Detection Sensor Module is a compact electronic device designed to accurately measure the level of water in various settings. Utilizing advanced sensor technology, it detects the presence or absence of water and provides real-time feedback to the connected system. With its simple yet efficient design, this module offers reliable performance and easy integration into a wide range of applications.\nApplications:\nThis versatile sensor module finds extensive use across different industries and scenarios. It is commonly employed in automated irrigation systems for agriculture, where it ensures optimal water usage by monitoring soil moisture levels. Additionally, it plays a crucial role in industrial processes such as water treatment plants, where maintaining precise water levels is essential for operational efficiency. Furthermore, it's utilized in household appliances like washing machin

In [26]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("What is the use of RC522 RFID Module in an IoT kit? Write the description, applications and technical specifications.")


print("Context:\n\n", result['context'],"\n\n")
print("Output:\n\n", result['text'],"\n\n")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Context:

 [Document(page_content='RC522 RFID Module:\nDescription:\nThe RC522 RFID Module is a compact and versatile device designed for RFID (Radio Frequency Identification) applications. It utilizes the popular RC522 chip, providing reliable and efficient RFID communication capabilities. With its small form factor and easy-to-use interface, this module is suitable for a wide range of projects and applications.\nApplications:\nThe RC522 RFID Module finds use in various applications across industries. It can be employed in access control systems, allowing authorized personnel to gain entry through RFID-enabled keycards or tags. Additionally, it is utilized in inventory management systems to track and manage assets efficiently. Other common applications include electronic toll collection, product authentication, and automated payment systems.\nTechnical Specification:\nOperating Voltage: 3.3V\nOperating Frequency: 13.56 MHz\nCommunication Interface: SPI (Serial Peripheral Interface)\nM